<a href="https://colab.research.google.com/github/Pnwankwo2/neural-network-challenge-1/blob/main/student_loans_with_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Student Loan Risk with Deep Learning

In [4]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path
from tensorflow.keras.optimizers import Adam
import os
from tensorflow.keras.models import save_model


---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [5]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [6]:
# Review the data types associated with the columns
loans_df.dtypes

,0
payment_history,float64
location_parameter,float64
stem_degree_score,float64
gpa_ranking,float64
alumni_success,float64
study_major_code,float64
time_to_completion,float64
finance_workshop_score,float64
cohort_ranking,float64
total_loan_score,float64


In [7]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

,count
credit_ranking,
1,855
0,744


### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [8]:
# Define the target set y using the credit_ranking column
y = loans_df['credit_ranking']

# Display the first few rows of the target set
print(y.head())

# Display a sample of y
y_sample = y.sample(10, random_state=42)  # Random sample of 10 values for reproducibility
y_sample

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64


,credit_ranking
803,1
124,0
350,1
682,0
1326,1
976,0
1493,0
706,0
613,0
1587,1


In [9]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns=['credit_ranking'])

# Review the features DataFrame
X.head()

# Display a sample of X
X_sample = X.sample(10, random_state=42)  # Random sample of 10 values for reproducibility
X_sample


,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
803,7.7,0.56,0.08,2.50,0.114,14.0,46.0,0.99710,3.24,0.66,9.6
124,7.8,0.50,0.17,1.60,0.082,21.0,102.0,0.99600,3.39,0.48,9.5
350,10.7,0.67,0.22,2.70,0.107,17.0,34.0,1.00040,3.28,0.98,9.9
682,8.5,0.46,0.31,2.25,0.078,32.0,58.0,0.99800,3.33,0.54,9.8
1326,6.7,0.46,0.24,1.70,0.077,18.0,34.0,0.99480,3.39,0.60,10.6
976,7.2,0.41,0.30,2.10,0.083,35.0,72.0,0.99700,3.44,0.52,9.4
1493,7.7,0.54,0.26,1.90,0.089,23.0,147.0,0.99636,3.26,0.59,9.7
706,7.0,0.78,0.08,2.00,0.093,10.0,19.0,0.99560,3.40,0.47,10.0
613,8.2,0.39,0.38,1.50,0.058,10.0,29.0,0.99620,3.26,0.74,9.8
1587,5.8,0.61,0.11,1.80,0.066,18.0,28.0,0.99483,3.55,0.66,10.9


### Step 3: Split the features and target sets into training and testing datasets.


In [10]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [11]:
# Create a StandardScaler instance
scaler = StandardScaler()

x_train = scaler.fit_transform(X_train)
x_test_scaled = scaler.transform(X_test)

# Fit the scaler to the features training dataset
scaler.fit(x_train)


StandardScaler()

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [12]:
# Define the the number of inputs (features) to the model
number_of_features = X_train.shape[1]
#print(f"Number of input features to the model: {number_of_features}")

# Review the number of features
print(f"Number of input features to the model: {number_of_features}")

Number of input features to the model: 11


In [13]:
# Define the number of input features
# Initialize the model
model = Sequential()

input_features = X_train.shape[1]

# Define the number of hidden nodes for the second hidden layer
model.add(Dense(units=32, activation='relu'))

# Define the number of neurons in the output layer
model.add(Dense(units=1, activation='sigmoid'))


In [14]:
# Create the Sequential model instance

input_dim = 11
output_dim = 10

# Add the first hidden layer

model.add(Dense(units=64, activation='relu', input_shape=(input_dim,)))


# Add the second hidden layer
model.add(Dense(units=32, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(units=1, activation='sigmoid'))
#model.add(Dense(units=64, activation='relu', input_dim=input_dim))
#model.add(Dense(units=10, activation='softmax'))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
# Display the Sequential model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [16]:
# Compile the Sequential model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Fit the model using 50 epochs and the training data
history = model.fit(
    X_train,        # Input features
    y_train,        # Target labels
    epochs=50,      # Number of epochs
    batch_size=32,  # Number of samples per gradient update
    validation_split=0.2,  # Fraction of training data to be used as validation
    verbose=1       # Verbosity mode (0 = silent, 1 = progress bar, 2 = one line per epoch)
)


Epoch 1/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.5025 - loss: 0.6933 - val_accuracy: 0.5273 - val_loss: 0.6917
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5454 - loss: 0.6904 - val_accuracy: 0.5273 - val_loss: 0.6918
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5380 - loss: 0.6904 - val_accuracy: 0.5273 - val_loss: 0.6924
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5387 - loss: 0.6904 - val_accuracy: 0.5273 - val_loss: 0.6917
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5611 - loss: 0.6883 - val_accuracy: 0.5273 - val_loss: 0.6918
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5499 - loss: 0.6889 - val_accuracy: 0.5273 - val_loss: 0.6923
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5441 - loss: 0.6893 - val_accuracy: 0.5273 - val_loss: 0.6921
Epoch 8/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5569 - loss: 0.6875 - val_accuracy: 0.5273 - val_loss

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [18]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
results = model.evaluate(X_test, y_test, batch_size=128, verbose=1)
test_loss, test_accuracy = results


# Display the model loss and accuracy results
print(f"Test Loss: {results[0]}")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7182 - loss: 0.5705  
Test Loss: 0.5867834687232971
Test Loss: 0.5868
Test Accuracy: 0.7000


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [27]:
# Set the model's file path
import os

# Define the directory path
directory = 'Patrick Nwankwo/Desktop/CHALLENGES-UPENN-VIRT-AI-PT-07-2024-U-LOLC'

# Create the directory if it doesn't exist
os.makedirs(directory, exist_ok=True)

# Define the file name
file_name = 'student_loans.keras'

# Combine directory and file name to create the full file path
file_path = os.path.join(directory, file_name)

# Save the model to the specified file path
model.save(file_path)



---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [28]:
# Set the model's file path

os.makedirs(directory, exist_ok=True)
directory = 'Patrick Nwankwo/Desktop/CHALLENGES-UPENN-VIRT-AI-PT-07-2024-U-LOLC'
file_name = 'student_loans.keras'
# Load the model to a new object
model = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [29]:
# Make predictions with the test data
from tensorflow.keras.models import load_model # Import load_model


model = load_model('Patrick Nwankwo/Desktop/CHALLENGES-UPENN-VIRT-AI-PT-07-2024-U-LOLC/student_loans.keras')


# Display a sample of the predictions
predictions = model.predict(X_test)
predictions_df = pd.DataFrame(predictions, columns=['Prediction'])
predictions_df.head()


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


,Prediction
0,0.564733
1,0.316932
2,0.763626
3,0.626480
4,0.828408


In [35]:
# Save the predictions to a DataFrame and round the predictions to binary results
binary_predictions = (predictions >= 0.5).astype(int)
binary_predictions_df = pd.DataFrame(binary_predictions, columns=['Prediction'])
binary_predictions_df.head()
results_df = pd.DataFrame({
    'Sample_ID': X_test.index,  # Replace with your actual identifier if available
    'Prediction': binary_predictions.flatten() # Flatten binary_predictions to 1-D
})

### Step 4: Display a classification report with the y test data and predictions

In [40]:
# Print the classification report with the y test data and predictions
from sklearn.metrics import classification_report
report = classification_report(y_test, binary_predictions)
print(report)


              precision    recall  f1-score   support

           0       0.67      0.75      0.71       154
           1       0.74      0.66      0.69       166

    accuracy                           0.70       320
   macro avg       0.70      0.70      0.70       320
weighted avg       0.70      0.70      0.70       320



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.


1.   List item

1.   List item
2.   List item


2.   List item


3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.